<p style="border: 1px solid #e7692c; border-left: 15px solid #e7692c; padding: 10px; text-align:justify;">
    <strong style="color: #e7692c">Tip.</strong> <a style="color: #000000;" href="https://nbviewer.jupyter.org/github/PacktPublishing/Hands-On-Computer-Vision-with-TensorFlow-2/blob/master/ch9/ch9_nb3_train_model.ipynb" title="View with Jupyter Online">Click here to view this notebook on <code>nbviewer.jupyter.org</code></a>. 
    <br/>These notebooks are better read there, as Github default viewer ignores some of the formatting and interactive content.
    </p>

<table style="font-size: 1em; padding: 0; margin: 0;">
    <tr style="vertical-align: top; padding: 0; margin: 0;">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #363636; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #e7692c;">Hands-on</span> Computer Vision with TensorFlow 2</span><br/>by <em>Eliot Andres</em> & <em>Benjamin Planche</em> (Packt Pub.)</strong><br/><br/>
        <strong>> Chapter 9: Performance and running on mobile</strong><br/>
    </p>

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #e7692c;">Notebook 3:</small><br/>Training a model and converting it for mobile devices</h1>
<br/>
<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    In this chapter, we covered how to convert and run a model on mobile.
<br/><br/>
    This notebooks trains a model to recognize face expressions and converts it to CoreML, TFLite and TensorFlow.js
</p>
<br/>

<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    <strong> Requirements </strong>
<br/><br/>
    To run this notebook, you need to download the <a href="https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data">FER dataset</a> and extract it. When done, change the `BASE_PATH` variable to point to the dataset folder.
</p>
<br/>
<p style="border-left: 15px solid #e7692c; padding: 0 10px; text-align:justify;">
    <strong style="color: #e7692c;">Tip.</strong> The notebooks shared on this git repository illustrate some of notions from the book "<em><strong>Hands-on Computer Vision with TensorFlow 2</strong></em>" written by Eliot Andres and Benjamin Planche and published by Packt. If you enjoyed the insights shared here, <strong>please consider acquiring the book!</strong>
<br/><br/>
The book provides further guidance for those eager to learn about computer vision and to harness the power of TensorFlow 2 and Keras to build performant recognition systems for object detection, segmentation, video processing, smartphone applications, and more.</p>
        </td>
        <td style="vertical-align: top; padding: 0; margin: 0; width: 255px;">
    <a href="https://www.packtpub.com" title="Buy on Packt!">
        <img src="../banner_images/book_cover.png">
    </a>
    <p style="background: #e7692c; color:#ffffff; padding: 10px; text-align:justify;"><strong>Leverage deep learning to create powerful image processing apps with TensorFlow 2 and Keras. <br/></strong>Get the book for more insights!</p>
    <ul style="height: 32px; white-space: nowrap; text-align: center; margin: 0px; padding: 0px; padding-top: 10px;">
    <li style="display: inline-block; height: 100%; vertical-align: middle; float: left; margin: 5px; padding: 0px;">
        <a href="https://www.packtpub.com" title="Get your Packt book!">
        <img style="vertical-align: middle; max-width: 75px; max-height: 32px;" src="../banner_images/logo_packt.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block; height: 100%; vertical-align: middle; float: left; margin: 5px; padding: 0px;">
        <a href="https://www.packtpub.com" title="Get the book on O'Reilly Safari!">
        <img style="vertical-align: middle; max-width: 75px; max-height: 32px;" src="../banner_images/logo_oreilly.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block; height: 100%; vertical-align: middle; float: left; margin: 5px; padding: 0px;">
        <a href="https://www.packtpub.com" title="Get the book on Amazon!">
        <img style="vertical-align: middle; max-width: 75px; max-height: 32px;" src="../banner_images/logo_amazon.png" width="75px">
        </a>
    </li>
    </ul>
        </td>
        </tr>
        </table>

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf

tf.__version__

'2.0.0-dev20190428'

## Set constants and parameters


In [2]:
# Download data here: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data
DATASET_PATH = '../../backup_book/ch9/fer2013/fer2013.csv'

IMAGE_SIZE = (48, 48)
INPUT_SHAPE = IMAGE_SIZE + (1,)
EMOTIONS = ["angry", "disgust", "scared",
            "happy", "sad", "surprised", "neutral"]
CALLBACK_PATIENCE = 50
BATCH_SIZE = 32
NUM_EPOCHS = 110
VALIDATION_SPLIT = .2
NUM_CLASSES = len(EMOTIONS)
L2_REGULARIZATION = 0.01

## Build the model

In [3]:
input_tensor = Input(shape=INPUT_SHAPE)
model = MobileNet(input_tensor=input_tensor, alpha=1.0,
                    include_top=False, weights=None)

output = tf.keras.layers.Reshape((1024,))(model.output)
output = tf.keras.layers.Dense(7, activation='softmax')(output)
model = tf.keras.Model(model.input, output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 49, 49, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 24, 24, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 24, 24, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 24, 24, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 24, 24, 32)        128   

## Load data

In [5]:
def load_fer2013():
    data = pd.read_csv(DATASET_PATH)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), IMAGE_SIZE)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    x = x - 0.5
    x = x * 2.0
    return x

data_generator = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    horizontal_flip=True)

# loading dataset
faces, emotions = load_fer2013()
faces = preprocess_input(faces)


## Train model

In [6]:
regularization = tf.keras.regularizers.l2(L2_REGULARIZATION)

early_stop = EarlyStopping('val_loss', patience=CALLBACK_PATIENCE)
reduce_lr = ReduceLROnPlateau(
    'val_loss', factor=0.1, patience=int(CALLBACK_PATIENCE/4), verbose=1)
tensorboard = TensorBoard('./logs')
callbacks = [early_stop, reduce_lr, tensorboard]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions, test_size=0.2)

model.fit_generator(data_generator.flow(xtrain, ytrain, BATCH_SIZE),
                    steps_per_epoch=len(xtrain) / BATCH_SIZE,
                    epochs=NUM_EPOCHS, verbose=1, callbacks=callbacks,
                    validation_data=(xtest, ytest))

Epoch 1/110
  1/897 [..............................] - ETA: 2:16:38 - loss: 2.4219 - accuracy: 0.1250

W0502 17:00:05.368174 139873498351360 callbacks.py:238] Method (on_train_batch_end) is slow compared to the batch update (0.111346). Check your callbacks.


898/897 [==============================] - 34s 38ms/step - loss: 1.8895 - accuracy: 0.2381 - val_loss: 1.9825 - val_accuracy: 0.2403
Epoch 2/110
898/897 [==============================] - 21s 24ms/step - loss: 1.8195 - accuracy: 0.2617 - val_loss: 1.7785 - val_accuracy: 0.2657
Epoch 3/110
898/897 [==============================] - 21s 24ms/step - loss: 1.7751 - accuracy: 0.2925 - val_loss: 1.8008 - val_accuracy: 0.3476
Epoch 4/110
898/897 [==============================] - 21s 24ms/step - loss: 1.7257 - accuracy: 0.3233 - val_loss: 1.6124 - val_accuracy: 0.3817
Epoch 5/110
898/897 [==============================] - 22s 24ms/step - loss: 1.6473 - accuracy: 0.3591 - val_loss: 1.6166 - val_accuracy: 0.3948
Epoch 6/110
898/897 [==============================] - 22s 24ms/step - loss: 1.6021 - accuracy: 0.3778 - val_loss: 2.3959 - val_accuracy: 0.3362
Epoch 7/110
898/897 [==============================] - 22s 24ms/step - loss: 1.5703 - accuracy: 0.3956 - val_loss: 1.5281 - val_accuracy: 0.42

KeyboardInterrupt: 

## Convert to CoreML

In [7]:
import tfcoreml as tf_converter
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.tools import freeze_graph
input_saved_model_dir = "./saved_model"

tf.keras.experimental.export_saved_model(
    model, saved_model_path=input_saved_model_dir, serving_only=False)


output_node_name = 'dense/Softmax'
input_binary = False
input_saver_def_path = False
restore_op_name = None
filename_tensor_name = None
clear_devices = True
input_meta_graph = False
checkpoint_path = None
input_graph_filename = None
saved_model_tags = tag_constants.SERVING


freeze_graph.freeze_graph(input_graph_filename, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          'frozen_model.pb', clear_devices, "", "", "",
                          input_meta_graph, input_saved_model_dir,
                          saved_model_tags)


tf_converter.convert('frozen_model.pb',
                     'mobilenet.mlmodel',
                     class_labels=EMOTIONS,
                     image_input_names=['input_1:0'],
                     output_feature_names=[output_node_name + ':0'],
                     red_bias=-1,
                     green_bias=-1,
                     blue_bias=-1,
                     image_scale=1/127.5,
                     is_bgr=False)

W0502 17:18:53.262634 139873498351360 __init__.py:118] TensorFlow version 2.0.0-dev20190428 detected. Last version known to be fully compatible is 1.12.0 .
W0502 17:18:58.839174 139873498351360 deprecation.py:323] From /root/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0502 17:18:58.839908 139873498351360 tf_logging.py:161] Export includes no default signature!
W0502 17:19:01.555757 139873498351360 tf_logging.py:161] Export includes no default signature!


NameError: name 'output_node_names' is not defined

## Convert to TFLite

In [16]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
## Or from a SavedModel
# converter = tf.lite.TFLiteConverter('./saved_model')

tflite_model = converter.convert()
open("result.tflite", "wb").write(tflite_model)



12822216

## Convert to TFJS

In [ ]:
# Convert in the current environment
import sys
!{sys.executable} tensorflowjs_converter --input_format=tf_saved_model saved_model my-tfjs --output_format tfjs_graph_model